_____

<table align="left" width=100%>
    <td>
        <div style="text-align: center;">
          <img src="./images/bar.png" alt="entidades financiadoras"/>
        </div>
    </td>
    <td>
        <p style="text-align: center; font-size:24px;"><b>Introduction to Data Science</b></p>
        <p style="text-align: center; font-size:18px;"><b>Master in Electrical and Computer Engineering</b></p>
        <p style="text-align: center; font-size:14px;"><b>Pedro Cardoso (pcardoso@ualg.pt)</b></p>
    </td>
</table>

_____

In [ ]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()
db = client.sensorsDB

## Query an array
Let us see some examples of how to make query operations on array fields using the db.collection.find(). 
The examples on this page use a `sensors_location` collection. 

To populate the `sensors_location` collection:

In [ ]:
data = [{
            'location_name': 'window @ lab 1', 
            'sensors': [ 'Temperature', 'Humidity']
       }, 
        {
            'location_name': 'door @ lab 1', 
            'sensors': [ 'IR', 'Proximity', 'Accelerometer']
       }, {
            'location_name': 'window @ lab 2', 
            'sensors': [ 'Temperature', 'Humidity', 'Pressure']
       }, 
        {
            'location_name': 'door @ lab 2', 
            'sensors': [ 'IR', 'Proximity']
       },
        {
            'location_name': 'N wall @ lab 3',
            'sensors': [ 'Gas', 'Smoke']
       },
        {
            'location_name': 'S wall @ lab 3', 
            'sensors': [ 'Smoke']
       }]

db.sensors_location.insert_many(data)
    

### Match an array
To specify equality condition on an array, use the query document `{ <field>: <value> }` where `<value>` is the exact array to match, including the order of the elements.

In [ ]:
list(db.sensors_location.find({'sensors':[ 'Gas', 'Smoke']}))

remember, we must observe the order

In [ ]:
list(db.sensors_location.find({'sensors':[ 'Smoke', 'Gas']}))

If the order is irrelevant, you can use the `$all` operator to match an array that contains all elements specified in the query. However, `$all` does not require an exact match of elements or order. For example

In [ ]:
list(db.sensors_location.find({'sensors':{'$all':[ 'Gas','Smoke']}}))

If you want those elements exactly, you can use the `$size` operator to match an array with the specified number of elements.

In [ ]:
list(db.sensors_location.find({'sensors':{'$size':2}, 'sensors':{'$all':[ 'Gas','Smoke']}}))

### Query an array for an element
To query if the array field contains at least one element with the specified value, use the filter `{ <field>: <value> }` where `<value>` is the element value.

In [ ]:
list(db.sensors_location.find({'sensors': 'Humidity'}))

Or locations with sensors with names greater than 'P' (why? because...)

In [ ]:
list(db.sensors_location.find({'sensors': { '$gte':'P'}}))

#### Query an array with compound Filter conditions on the array elements
Or locations with sensors with names between 'A' and 'H' (why? because...)

In [ ]:
list(db.sensors_location.find({'sensors': { '$gte':'A', '$lte':'H'}}))

# Query for an element by the array index position

To query for an element at a specific index position, use the dot notation `<arrayField>.<index>` in the query filter. The index value is zero-based.

In [ ]:
list(db.sensors_location.find({'sensors.1': 'Smoke'}))

### Query an array by array length

do we have locations with 3 sensors?

In [ ]:
list(db.sensors_location.find({'sensors': {'$size':3}}))

## Query an array of embedded documents

As seen, it is usual to have arrays of embedded documents. Let us see some examples of how to make query operations on array fields using the db.collection.find().



In [ ]:
import numpy as np

np.random.seed(1)

def sample(n=10, m=100): # returns a sample of n values from [0, 1, ...., m-1]
    return list(map(int,np.random.choice(m, n, replace=True).astype(int)))

data = [{
            'location_name': 'window @ lab 1', 
            'sensors': [ {'type': 'Temperature', 
                          'readings': sample()
                         },
                        {'type': 'Humidity', 
                          'readings': sample()
                        }]
       },{
            'location_name': 'door @ lab 1', 
            'sensors': [ {'type': 'IR', 
                          'readings': sample()},
                        {'type': 'Proximity', 
                          'readings': sample()
                        },
                        {'type': 'Accelerometer', 
                          'readings': sample()
                        }]
       },{
            'location_name': 'window @ lab 2', 
            'sensors': [ {'type': 'Temperature', 
                          'readings': sample()},
                        {'type': 'Humidity', 
                          'readings': sample()
                        },
                        {'type': 'Pressure', 
                          'readings': sample()
                        }]
       },{
            'location_name': 'door @ lab 2', 
            'sensors': [ {'type': 'IR', 
                          'readings': sample()},
                        {'type': 'Proximity', 
                          'readings': sample()
                        }]
       }]

db.sensors_data.insert_many(data)

In [ ]:
pprint(data)

All readings from humidity sensors

In [ ]:
for doc in db.sensors_data.find({'sensors.type':'Humidity'}):
    pprint(doc)

All humidity sensors with readings equal to 82

In [ ]:
query = {
        'sensors': { 
            '$elemMatch' :{
                'type':'Humidity', 
                'readings': 82
            }
        }
    }

for doc in db.sensors_data.find(query):
    pprint(doc)

All humidity sensors with readings greater than 90

In [ ]:
query = {
        'sensors': { 
            '$elemMatch' :{
                'type':'Humidity', 
                'readings': {'$gt': 90}
            }
        }
    }

for doc in db.sensors_data.find(query):
    pprint(doc)